d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Reading and Writing to Synapse

## Learning Objectives
By the end of this lesson, you should be able to:
* Describe the connection architecture of Synapse and Spark
* Configure a connection between Databricks and Synapse
* Read data from Synapse
* Write data to Synapse

### Azure Synapse
- leverages massively parallel processing (MPP) to quickly run complex queries across petabytes of data
- PolyBase T-SQL queries

## Synapse Connector
- uses Azure Blob Storage as intermediary
- uses PolyBase in Synapse
- enables MPP reads and writes to Synapse from Azure Databricks

Note: The Synapse connector is more suited to ETL than to interactive queries. For interactive and ad-hoc queries, data should be extracted into a Databricks Delta table.

```
                           ┌─────────┐
      ┌───────────────────>│ STORAGE │<──────────────────┐
      │ Storage acc key /  │ ACCOUNT │ Storage acc key / │
      │ Managed Service ID └─────────┘ OAuth 2.0         │
      │                         │                        │
      │                         │ Storage acc key /      │
      │                         │ OAuth 2.0              │
      v                         v                 ┌──────v────┐
┌──────────┐              ┌──────────┐            │┌──────────┴┐
│ Synapse  │              │  Spark   │            ││ Spark     │
│ Analytics│<────────────>│  Driver  │<───────────>| Executors │
└──────────┘  JDBC with   └──────────┘ Configured  └───────────┘
              username &               in Spark
              password
```

## SQL DW Connection

Three connections are made to exchange queries and data between Databricks and Synapse
1. **Spark driver to Synapse**
   - the Spark driver connects to Synapse via JDBC using a username and password
2. **Spark driver and executors to Azure Blob Storage**
   - the Azure Blob Storage container acts as an intermediary to store bulk data when reading from or writing to Synapse
   - Spark connects to the Blob Storage container using the Azure Blob Storage connector bundled in Databricks Runtime
   - the URI scheme for specifying this connection must be wasbs
   - the credential used for setting up this connection must be a storage account access key
   - the account access key is set in the session configuration associated with the notebook that runs the command
   - this configuration does not affect other notebooks attached to the same cluster. `spark` is the SparkSession object provided in the notebook
3. **Synapse to Azure Blob Storage**
   - Synapse also connects to the Blob Storage container during loading and unloading of temporary data
   - set `forwardSparkAzureStorageCredentials` to true
   - the forwarded storage access key is represented by a temporary database scoped credential in the Synapse instance
   - Synapse connector creates a database scoped credential before asking Synapse to load or unload data
   - then it deletes the database scoped credential once the loading or unloading operation is done.

## Configuration

### Create Azure Blob Storage
Follow these steps to [create an Azure Storage Account](https://docs.microsoft.com/en-us/azure/storage/common/storage-quickstart-create-account?tabs=azure-portal#regenerate-storage-access-keys) and Container. The Synapse connector will use a [Shared Key](https://docs.microsoft.com/en-us/rest/api/storageservices/authorize-with-shared-key) for authorization. Be sure to make note of the **Storage Account Name**, the **Container Name**, and the **Access Key** while working through these steps:

1. Access the Azure Portal
2. Create a New Resource
3. Create a Storage account
4. Make sure to specify the correct *Resource Group* and *Region*. Use any unique string as the  for the **Storage Account Name**
5. Access Blobs
6. Create a New Container using any unique string for the **Container Name**
7. Retrieve the primary **Access Key** for the new Storage Account

In the cell below, enter the **Storage Account Name**, the **Container Name**, and the **Access Key**.

In [0]:
storageAccount = dbutils.secrets.get(scope="demo", key="storageaccount")
containerName = "polybase"
accessKey = dbutils.secrets.get(scope="demo", key="storagekey")

spark.conf.set(f"fs.azure.account.key.{storageAccount}.blob.core.windows.net", accessKey)

## Configuration

### Create an Azure Synapse Instance
Follow these steps to [create an Azure Storage Account](https://docs.microsoft.com/en-us/azure/storage/common/storage-quickstart-create-account?tabs=azure-portal#regenerate-storage-access-keys) and Container. The Synapse connector will use a [Shared Key](https://docs.microsoft.com/en-us/rest/api/storageservices/authorize-with-shared-key) for authorization. Be sure to make note of the **Storage Account Name**, the **Container Name**, and the **Access Key** while working through these steps:

1. Access the Azure Portal
2. Create a New Resource
3. Create a Synapse instance using these attributes:
   - Use any string for the **Database Name**
   - Select "Sample" as the Source
   - Select an existing or create a new SQL Server
5. Access the new Synapse instance
6. Select Query Editor (preview) and enter the proper credentials
7. Run these two queries:
   - Create a Master Key in the SQL DW. This facilitates the SQL DW connection

     `CREATE MASTER KEY ENCRYPTION BY PASSWORD = 'CORRECT-horse-battery-staple';`

   - Use a CTAS to create a staging table for the Customer Table. This query will create an empty table with the same schema as the Customer Table.

     ```
     CREATE TABLE dbo.DimCustomerStaging
     WITH
     ( DISTRIBUTION = ROUND_ROBIN, CLUSTERED COLUMNSTORE INDEX )
     AS
     SELECT  *
     FROM dbo.DimCustomer
     WHERE 1 = 2
     ;
     ```
7. Access Connection Strings.
8. Select JDBC and copy the **JDBC URI**.

In the cell below, we'll access the JDBC URI stored securely in Key Vault.
Note that the table we will be using has already been defined.

In [0]:
tableName = "dbo.DimCustomer"
jdbcURI = dbutils.secrets.get(scope="demo", key="jdbc")

## Read from the Customer Table

Next, use the Synapse Connector to read data from the Customer Table.

Use the read to define a tempory table that can be queried.

Note:

- the connector uses a caching directory on the Azure Blob Container.
- `forwardSparkAzureStorageCredentials` is set to `true` so that the Synapse instance can access the blob for its MPP read via Polybase

In [0]:
cacheDir = f"wasbs://{containerName}@{storageAccount}.blob.core.windows.net/cacheDir"

customerDF = (spark.read
  .format("com.databricks.spark.sqldw")
  .option("url", jdbcURI)
  .option("tempDir", cacheDir)
  .option("forwardSparkAzureStorageCredentials", "true")
  .option("dbTable", tableName)
  .load())

customerDF.createOrReplaceTempView("customer_data")

Use SQL queries to count the number of rows in the Customer table and to display table metadata.

In [0]:
%sql
select count(*) from customer_data

In [0]:
%sql
describe customer_data

Note that `CustomerKey` and `CustomerAlternateKey` use a very similar naming convention.

In [0]:
%sql
select CustomerKey, CustomerAlternateKey from customer_data limit 10;

In a situation in which we may be merging many new customers into this table, we can imagine that we may have issues with uniqueness with regard to the `CustomerKey`. Let us redefine `CustomerAlternateKey` for stronger uniqueness using a [UUID](https://en.wikipedia.org/wiki/Universally_unique_identifier).

To do this we will define a UDF and use it to transform the `CustomerAlternateKey` column. Once this is done, we will write the updated Customer Table to a Staging table.

**Note:** It is a best practice to update the Synapse instance via a staging table.

In [0]:
import uuid

from pyspark.sql.types import StringType
from pyspark.sql.functions import udf

uuidUdf = udf(lambda : str(uuid.uuid4()), StringType())
customerUpdatedDF = customerDF.withColumn("CustomerAlternateKey", uuidUdf())
display(customerUpdatedDF)

### Use the Polybase Connector to Write to the Staging Table

In [0]:
(customerUpdatedDF.write
  .format("com.databricks.spark.sqldw")
  .mode("overwrite")
  .option("url", jdbcURI)
  .option("forward_spark_azure_storage_credentials", "true")
  .option("dbtable", tableName + "Staging")
  .option("tempdir", cacheDir)
  .save())

## Read and Display Changes from Staging Table

In [0]:
customerTempDF = (spark.read
  .format("com.databricks.spark.sqldw")
  .option("url", jdbcURI)
  .option("tempDir", cacheDir)
  .option("forwardSparkAzureStorageCredentials", "true")
  .option("dbTable", tableName + "Staging")
  .load())

customerTempDF.createOrReplaceTempView("customer_temp_data")

In [0]:
%sql
select CustomerKey, CustomerAlternateKey from customer_temp_data limit 10;


-sandbox
&copy; 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>